# Data Processing

## Import modules

In [ ]:
import os
import numpy as np
import pandas as pd
import re
import time

## Event filtration
Events column is a bit chaotic as events are written manually and for many registers it is not standarized. In here I write a list with all events written in there, from which I will manually select those meaning `MOV` (movement), `LMA` (larynge mask), `INT` (intubation) and drugs, by searching if certain characters are in the event. Ideally we should use natural lenguage processing for this. 

In [ ]:
events=[]
no_events=[]

#list with all registers
my_path=r'C:\Users\joana\OneDrive\Escriptori\TFG\data\Datos_Clinic'
file_names=os.listdir(my_path)

#Iterate over each register
for i in range(len(file_names)):
    
    data=pd.read_excel('Datos_Clinic/'+file_names[i],'Hoja1')
    
    #Check if column Events exists
    if 'Events' in data.columns:
        
        #Iterate over column Events
        for event in data['Events']:
            
            #Append all events to a list except if it is already there
            if event not in events:
                events.append(event)
                
    #If column Events is missing append file name to a list
    else:
        no_events.append(file_names[i])

        
#Save lists  
pd.DataFrame(no_events).to_csv('events//no_events.csv',index=False,header=False,encoding='latin1')

#Avoid saving NaNs by selecting only strings
events=[event for event in events if type(event)==str]
pd.DataFrame(events).to_csv('events\events.csv',index=False,header=False,encoding='latin1')

## Data wrangling
For each register, `MOV`, `INT`, `LMA` and drugs columns are created and filled. Values of `128` in conox variables are set as NaNs, as they mean the machine is recalibrating. Missing values are then interpolated linearly. registers are cut before `CpPROPO` or `CpREMI` reaches 1, as it means anaesthesia is starting. Time column is reindexed so it starts at 1. For registers lacking frequencies columns they are created and set to NaN. Finally, columns of interest are selected and saved.

In [ ]:
start_time=time.time()

#lists of events meaning LMA, MOV and INT
lma=pd.read_csv('events/lma.csv',encoding='latin1',header=None,index_col=False)[0].tolist()
movs=pd.read_csv('events/movs.csv',encoding='latin1',header=None,index_col=False)[0].tolist()
tub=pd.read_csv('events/tub.csv',encoding='latin1',header=None,index_col=False)[0].tolist()

#list that contains registers without events in it (therefore useless)
no_events=pd.read_csv('events/no_events.csv',encoding='latin1',header=None,index_col=False)[0].tolist()

#list with all registers with events
my_path=r'C:\Users\joana\OneDrive\Escriptori\TFG\Datos_Clinic'
file_names=[elem for elem in os.listdir(my_path) if elem not in no_events]

#Extract float or integer numbers in a string (dose)
def extract_dose(event): 
    
    #create empty list
    nums=[] 
    
    #find float number (comma and dot) and integers and append first number to appear (so that it doesn't add 5 in 1.5)
    nums.append(float(re.findall("[+-]?\d+\.\d+",event)[0]) if re.findall("[+-]?\d+\.\d+",event) else 0)
    nums.append(float(re.findall("[+-]?\d+\,\d+",event)[0].replace(',','.')) if re.findall("[+-]?\d+\,\d+",event) else 0)
    nums.append(float(re.findall("[+-]?\d+",event)[0]) if re.findall("[+-]?\d+",event) else 0)
    
    #return highest so that it neglects integers (1.5>1)
    return(max(nums)) 

    
#iterate over each raw register
for i in range(len(file_names)):
    
    #open data
    data=pd.read_excel('Datos_Clinic/'+file_names[i],'Hoja1')
    
    #initialize new separated events columns
    for col in ['MOV','LMA','INT','ROCURONIUM','EFEDRINE','ATROPINE','METHADONE','DROPERIDOL']:
        data[col]=0
    
    
    #iterate over each register
    for j in range(len(data)):
        
        #select event and check if it is string (otherwise nan -> no event)
        event=data['Events'][j]
        if type(event)!=str:
            continue
        
        #fill the new columns (MOV -> 1 or 0, drugs -> dose)
        if event in movs:
            data.at[j,'MOV']=1
        if 'rocu' in event.lower() and 'min' not in event.lower():
            data.at[j,'ROCURONIUM']=int(re.findall(r"\d+",event.replace('1O','10').replace('330','30'))[0]) if re.findall(r"\d+",event) else 0
        if 'efed' in event.lower() or 'ephe' in event.lower() and 'min' not in event.lower():
            data.at[j,'EFEDRINE']=int(re.findall(r"\d+",event)[0]) if re.findall(r"\d+",event) else 0
        if 'meta' in event.lower() or 'meth' in event.lower():
            if event in ['0.75mg droperidol 3 mg metadona','0.8mg droperidol 3mg metadona','.75 droperidol, 3 metadona']:
                num=3
            elif event in ['abans de la metadona, 2.5 neoxinina i 0.5 atropina','10 metadona puesta a 10:28:14']:
                num=0
            else:
                num=int((re.findall("[+-]?\d+",event)[0]) if re.findall("[+-]?\d+",event) else 0)
            data.at[j,'METHADONE']=num
        if 'atro' in event.lower() and 'min' not in event.lower():
            num=extract_dose(event.replace('0.06','0.6').replace('4,7','0,7'))
            if event=='abans de la metadona, 2.5 neoxinina i 0.5 atropina':
                num=0.5
            elif event=='4 mg prostignina + 1mg atropina':
                num=1
            data.at[j,'ATROPINE']=num
        if 'dro' in event.lower():
            num=extract_dose(event)
            num=0.75 if num>1.5 else num
            data.at[j,'DROPERIDOL']=num
            
    #iterate over each register
    for j in range(len(data)):
        
        #fill LMA or INT columns (cannot be both, cannot be more than once)
        if data['Events'][j] in lma:
            data.at[j,'LMA']=1
            break
        elif data['Events'][j] in tub:
            data.at[j,'INT']=1
            break
    
    #The data acquisition machine will recalibrate itself and the values of 128 will appear in qCON and qCONqNOX which does 
    #is not feasible as they cannot go above 100% -> converted to NaNs
    for col in ['qCON', 'qCONEMG', 'qCONBS', 'qCONSQI', 'qCONqNOX', 'qCONZneg','qCONZref', 'qCONZpos']:
        data.loc[data[col]==128, col] = np.nan
    
    #find first value where drug concentration is higher that 1 -> surgery start
    k=0
    for k in range(len(data)):
        if data['CpPROPO'][k]>1 or data['CpREMI'][k]>1:
            break
    
    #interpolate missing values, even if not perfect it is the best possible approach
    data_int=data.interpolate(method='linear')
    
    #cut all values before surgery start, reset index and reset Time(s)
    data_cut=data_int[k:]
    data_cut.reset_index(drop=True, inplace=True)
    data_cut.at[:,'Time(s)']=range(1,1+len(data_cut))
    
    #list containing names for all desired frequency columns (0Hz to 45Hz)
    freqs=[str(i)+'Hz' for i in range(46)]
    
    #there are a few registers without frequencies, for which empty frequency columns are created 
    if ~data_cut.columns.isin(freqs).any():
        for col in freqs:
            data_cut[col]=np.nan
    
    #select & order columns to be exported
    data_out=data_cut[['ID', 'Time(s)', 'ApproxRAppTime', 'SystTimeBIS', 'BIS', 'EMGBIS',
       'BSBIS', 'SQI09', 'CpREMI', 'CeREMI', 'InfVolREMI', 'InfRateREMI',
       'CpPROPO', 'CePROPO', 'InfVolPROPO', 'InfRatePROPO', 'Age',
       'Height', 'Weight', 'Gender', 'LBM', 'BSA', 'HR', 'NIBPsys',
       'NIBPdia', 'NIBPmean', 'RespiRate', 'SPO2',  'LMA','INT',
       'MOV',  'ROCURONIUM', 'ATROPINE',
       'EFEDRINE', 'METHADONE',  'Events', 
       'qCON', 'qCONEMG', 'qCONBS', 'qCONSQI', 'qCONqNOX', 'qCONZneg',
       'qCONZref', 'qCONZpos']+freqs]
    
    
    data_out.to_excel('Datos_Procesados\processed_data_'+str(int(data_out['ID'][0]))+'.xlsx',sheet_name='Hoja1',index=False)
    

print('took ',round((time.time()-start_time)/60),' mins')

## Missing columns & Mask separation
As we will focus on LMA registers, we want to create a list with these registers.

There are some registers which are missing columns. In here, I search for LMA registers that are missing columns in the region of interest (when the mask is introduced) in order to exclude them.

In [ ]:
start_time=time.time()

#List of registers folowing inclusion criteria
my_path=r'C:\Users\joana\OneDrive\Escriptori\TFG\data\Datos_Procesados'
file_names=os.listdir(my_path)

#Initialize empty list for registers having empty columns 
missing_cols=[]
lma_registers=[]

#Columns of interest
predictors=['BIS', 'EMGBIS',
'BSBIS', 'SQI09', 'CpREMI', 'CeREMI', 'InfVolREMI', 'InfRateREMI',
'CpPROPO', 'CePROPO', 'InfVolPROPO', 'InfRatePROPO', 'Age',
'Height', 'Weight', 'Gender', 'LBM', 'BSA', 'HR', 'NIBPsys',
'NIBPdia', 'NIBPmean', 'RespiRate', 'SPO2', 
'qCON', 'qCONEMG', 'qCONBS', 'qCONSQI', 'qCONqNOX']+[str(i)+'Hz' for i in range(31)]

#Iterate over each register
for reg in file_names:

    #Open file and select columns
    file_name='data\Datos_Procesados/'+reg
    df=pd.read_excel(file_name,'Hoja1')
    df_select=df[predictors]
    
    #initialize max and min columns
    if reg==file_names[0]:
        max_values=pd.DataFrame(df_select.min())
        min_values=pd.DataFrame(df_select.min())
    
    #update max and mon columns
    min_values=pd.concat([min_values,pd.DataFrame(df_select.min())],axis=1).min(axis=1)
    max_values=pd.concat([max_values,pd.DataFrame(df_select.min())],axis=1).max(axis=1)

    #check if it is a LMA register
    if df['LMA'].sum()==1:
        
        #append to lma list
        lma_registers.append(reg)
        
        #Fin region of interest (LMA) and cut dataframe
        k=0
        for k in range(len(df)):
            if df['LMA'][k]==1:
                break

        #select 45s before LMA and remove nans by rows
        df_start=df_select.iloc[(k-45):k,:]
        df_start=df_start[~np.isnan(df_start).any(axis=1)]
        
        #check if there are missing rows
        if len(df_start)!=45:
            missing_cols.append(reg)


#save list
pd.DataFrame(missing_cols).to_csv('events\\missing_columns.csv',index=False,header=False)
pd.DataFrame(lma_registers).to_csv('events\\reg_lma.csv',index=False,header=False)
maxmin=pd.concat([min_values,max_values],axis=1)
maxmin.columns=['min','max']
pd.DataFrame(maxmin).to_csv('events\\maxmin.csv')
        
print('took ',round((time.time()-start_time)/60),' mins')

## Classify registers
Classify registers according to output

In [ ]:
#function to shift the desired variable column by a desired number
def series_to_supervised(data,var,n_out=1):
    
    
    #turn data do pandas dataframe
    df=pd.DataFrame(data)
    cols=[df]
    
    
    #create shifted column
    shift_col=df[var].shift(-n_out)
    
    
    #add shifted column to dataframe
    cols.append(shift_col)
    df_shift=pd.concat(cols,axis=1)
    
    
    #rename column
    df_shift.columns=np.append(df.columns.values,var+' shift='+str(n_out))

    
    #return dataframe
    return df_shift

In [ ]:
#open LMA registers that follow inclusion criteria
registers=pd.read_csv('events/reg_lma.csv',header=None,index_col=False)[0].tolist()
missing_col=pd.read_csv('events/missing_columns.csv',header=None,index_col=False)[0].tolist()
registers=[reg for reg in registers if reg not in missing_col]

#initialise lists
reg_mov=[]
reg_nibp=[]
reg_bis=[]

#Columns of interest
predictors=['BIS', 'EMGBIS',
'BSBIS', 'SQI09', 'CpREMI', 'CeREMI', 'InfVolREMI', 'InfRateREMI',
'CpPROPO', 'CePROPO', 'InfVolPROPO', 'InfRatePROPO', 'Age',
'Height', 'Weight', 'Gender', 'LBM', 'BSA', 'HR', 'NIBPsys',
'NIBPdia', 'NIBPmean', 'RespiRate', 'SPO2', 
'qCON', 'qCONEMG', 'qCONBS', 'qCONSQI', 'qCONqNOX','MOV']+[str(i)+'Hz' for i in range(31)]

#Iterate over each register
for reg in registers:

    #Open file and select columns
    file_name='data\Datos_Procesados/'+reg
    df=pd.read_excel(file_name,'Hoja1')
    df_select=df[predictors]
    
    #Reframe for supervised learning (shift=45s) + remove MOV col (not really a predictor)
    df_reframed=series_to_supervised(df_select,'BIS',45)
    df_reframed=series_to_supervised(df_reframed,'MOV',45)
    df_reframed=series_to_supervised(df_reframed,'NIBPmean',45)
    
    #Fin region of interest (LMA) and cut dataframe
    k=0
    for k in range(len(df)):
        if df['LMA'][k]==1:
            break
    
    #classify registers
    df_start=df_reframed.iloc[(k-45):k,:]
    
    if (df_start.iloc[:,-3]>60).any():
        reg_bis.append(reg)
    
    if (df_start.iloc[:,-2]==1).any():
        reg_mov.append(reg)
        
    if (df_start.iloc[:,-1]<60).any():
        reg_nibp.append(reg)

#save list
pd.DataFrame(reg_bis).to_csv('events\\reg_bis.csv',index=False,header=False)
pd.DataFrame(reg_mov).to_csv('events\\reg_mov.csv',index=False,header=False)
pd.DataFrame(reg_nibp).to_csv('events\\reg_nibp.csv',index=False,header=False)

## Balance data and define train and test
First, data is balanced so that positive and negative group have the same amount of registers. To do so, for each group, negative registers are randomly selected to match the number of positive registers.

Then, randomly split the registers that fulfill the inclusion criteria (LMA and no missing columns) in train (80%) and test (20%) set.

In [ ]:
#Open list with register names
registers=pd.read_csv('events/reg_lma.csv',header=None,index_col=False)[0].tolist()
missing_col=pd.read_csv('events/missing_columns.csv',header=None,index_col=False)[0].tolist()
registers=[reg for reg in registers if reg not in missing_col]

reg_bis=pd.read_csv('events/reg_bis.csv',header=None,index_col=False)[0].tolist()
reg_mov=pd.read_csv('events/reg_mov.csv',header=None,index_col=False)[0].tolist()
reg_nibps=pd.read_csv('events/reg_nibp.csv',header=None,index_col=False)[0].tolist()

#BIS

#radnomly choose registers with bis lower than 60
reg_bis_neg=np.random.choice([reg for reg in registers if reg not in reg_bis],len(reg_bis),replace=False)

#create a mask
msk_bis = np.full(len(reg_bis),False)
inds=np.random.choice(np.arange(len(reg_bis)),size=round(len(reg_bis)*.8),replace=False)
msk_bis[inds]=True

#create lists with indexes
reg_train_bis=[reg for reg,tr in zip(reg_bis,msk_bis) if tr] + [reg for reg,tr in zip(reg_bis_neg,msk_bis) if tr]
reg_test_bis=[reg for reg,tr in zip(reg_bis,msk_bis) if not tr] + [reg for reg,tr in zip(reg_bis_neg,msk_bis) if not tr]

#Save sets
pd.DataFrame(reg_train_bis).to_csv('events\\reg_train_bis.csv',index=False,header=False)
pd.DataFrame(reg_test_bis).to_csv('events\\reg_test_bis.csv',index=False,header=False)

#MOV

#radnomly choose registers with bis lower than 60
reg_mov_neg=np.random.choice([reg for reg in registers if reg not in reg_mov],len(reg_mov),replace=False)

#create a mask
msk_mov = np.full(len(reg_mov),False)
inds=np.random.choice(np.arange(len(reg_mov)),size=round(len(reg_mov)*.8),replace=False)
msk_mov[inds]=True

#create lists with indexes
reg_train_mov=[reg for reg,tr in zip(reg_mov,msk_mov) if tr] + [reg for reg,tr in zip(reg_mov_neg,msk_mov) if tr]
reg_test_mov=[reg for reg,tr in zip(reg_mov,msk_mov) if not tr] + [reg for reg,tr in zip(reg_mov_neg,msk_mov) if not tr]

#Save sets
pd.DataFrame(reg_train_mov).to_csv('events\\reg_train_mov.csv',index=False,header=False)
pd.DataFrame(reg_test_mov).to_csv('events\\reg_test_mov.csv',index=False,header=False)

#NIBP

#radnomly choose registers with bis lower than 60
reg_nibp_neg=np.random.choice([reg for reg in registers if reg not in reg_nibp],len(reg_nibp),replace=False)

#create a mask
msk_nibp = np.full(len(reg_nibp),False)
inds=np.random.choice(np.arange(len(reg_nibp)),size=round(len(reg_nibp)*.8),replace=False)
msk_nibp[inds]=True

#create lists with indexes
reg_train_nibp=[reg for reg,tr in zip(reg_nibp,msk_nibp) if tr] + [reg for reg,tr in zip(reg_nibp_neg,msk_nibp) if tr]
reg_test_nibp=[reg for reg,tr in zip(reg_nibp,msk_nibp) if not tr] + [reg for reg,tr in zip(reg_nibp_neg,msk_nibp) if not tr]

#Save sets
pd.DataFrame(reg_train_nibp).to_csv('events\\reg_train_nibp.csv',index=False,header=False)
pd.DataFrame(reg_test_nibp).to_csv('events\\reg_test_nibp.csv',index=False,header=False)

## Prepare data to train the models
Data must be prepared to train a model such that it matches between registers. To do this data will be normalized between `0` and `1` using $scaled = \frac{value - min}{max - min}$ and the same moment of the operation will be used, that is, the 45s before LMA as input and the 45s after LMA as output. Output will be categorical, either 0 (good outcome) or 1 (bad outcome) for three different variables:
* MOV: whether the patient moves 
* BIS: whether there is a BIS above 60% (high state of consciousness)
* Mean arterial pressure: whether there is mean arterial pressure below 60mmHg (bad tissue perfusion)

If one of these variables for the patient is positive even if it is for just a second, we will consider positive for all 45 points (seconds). 

It is important to bear in mind that values are scaled so thresholds also have to be scaled. For BIS it is as simple as dividing by 100 (as it ranges from 0 to 100), for NIBP we must use the formula above.

In [ ]:
#Load test and train
reg_train_bis=pd.read_csv('events/reg_train_bis.csv',header=None,index_col=False)[0].tolist()
reg_test_bis=pd.read_csv('events/reg_test_bis.csv',header=None,index_col=False)[0].tolist()

reg_train_mov=pd.read_csv('events/reg_train_mov.csv',header=None,index_col=False)[0].tolist()
reg_test_mov=pd.read_csv('events/reg_test_mov.csv',header=None,index_col=False)[0].tolist()

reg_train_nibp=pd.read_csv('events/reg_train_nibp.csv',header=None,index_col=False)[0].tolist()
reg_test_nibp=pd.read_csv('events/reg_test_nibp.csv',header=None,index_col=False)[0].tolist()

In [ ]:
#function to turn output to categorical
def to_categorical(y_data,model):
    
    
    #turn data to pandas dataframe
    y_data=pd.DataFrame(y_data)
    
    
    #for BIS positive means above 60%
    if model=='BIS':
        
        
        #iterate over each patient (45s/patient chosen)
        for i in range(int(len(y_data)/45)):

            #if any point is above 60%BIS patient is set to 1
            if (y_data.iloc[i*45:(i+1)*45]>0.614).any()[0]:
                y_data.iloc[i*45:(i+1)*45]=1
            else:
                y_data.iloc[i*45:(i+1)*45]=0
    
    
    #for MOV positive means at least 1
    elif model=='MOV':
        
        
        #iterate over each patient (45s/patient chosen)
        for i in range(int(len(y_data)/45)):

            #if any point is mov 1 patient is set to 1
            if (y_data.iloc[i*45:(i+1)*45]==1).any()[0]:
                y_data.iloc[i*45:(i+1)*45]=1
            else:
                y_data.iloc[i*45:(i+1)*45]=0      
                
    
    #for NIBP positive means below 60
    elif model=='NIBPmean':
        
        
        #iterate over each patient (45s/patient chosen)
        for i in range(int(len(y_data)/45)):

            #if any point is above 60%BIS patient is set to 1
            if (y_data.iloc[i*45:(i+1)*45]<0.21348).any()[0]:
                y_data.iloc[i*45:(i+1)*45]=1
            else:
                y_data.iloc[i*45:(i+1)*45]=0      
        
    
    #return data as array
    y_data=np.asarray(y_data)
    y_data=y_data.astype(int)
    
    return y_data

In [ ]:
def data_processing(registers,variable):
    
    
    #list of predictors
    predictors=['BIS', 'EMGBIS',
    'BSBIS', 'SQI09', 'CpREMI', 'CeREMI', 'InfVolREMI', 'InfRateREMI',
    'CpPROPO', 'CePROPO', 'InfVolPROPO', 'InfRatePROPO', 'Age',
    'Height', 'Weight', 'LBM', 'BSA', 'HR', 'NIBPsys',
    'NIBPdia', 'NIBPmean', 'RespiRate', 'SPO2', 
    'qCON', 'qCONEMG', 'qCONBS', 'qCONSQI', 'qCONqNOX','MOV']+[str(i)+'Hz' for i in range(31)]

    
    #Open up maxmin from each column in order to manually scale from 0 to 1
    maxmin=pd.read_csv('events/max_min.csv',index_col=0)
    maxmin=maxmin.transpose(copy=True)[predictors]
    
    
    #For each patient
    for reg in registers:
        
        
        #Open up file and select desired columns
        file_name='data/Datos_Procesados/'+reg
        df=pd.read_excel(file_name,'Hoja1')
        df_select=df[predictors]
        
        
        #Normalize
        df_scaled=(df_select-maxmin.loc['min'])/(maxmin.loc['max']-maxmin.loc['min'])
        df_scaled=df_scaled[predictors]
        
        
        #Reframe for supervised learning (shift=45s) + remove MOV col (not really a predictor)
        df_reframed=series_to_supervised(df_scaled,variable,45)
        df_reframed.drop('MOV',axis=1, inplace=True)

        
        #Find laryngeal mask introduction point
        k=0
        for k in range(len(df)):
            if df['LMA'][k]==1:
                break
         
        
        #select 45s before intubation (so input is the 45s before and output is the 45s after)
        df_start=df_reframed.iloc[(k-45):k,:]
        
        
        
        #split X and y (first initialise then append)
        if reg==registers[0]:
            inp=df_start.values[:,:-1]
            out=df_start.values[:,-1]
        else:
            inp=np.concatenate((inp,df_start.values[:,:-1]))
            out=np.concatenate((out,df_start.values[:,-1]))
            
        
    #turn y to categorical
    out=to_categorical(out,variable)
    
    
    #return X and y
    return inp,out

In [ ]:
start_time=time.time()


#process X and y bis
train_X_bis,train_y_bis=data_processing(reg_train_bis,'BIS')
test_X_bis,test_y_bis=data_processing(reg_test_bis,'BIS')

#process X and y mov
train_X_mov,train_y_mov=data_processing(reg_train_mov,'MOV')
test_X_mov,test_y_mov=data_processing(reg_test_mov,'MOV')

#process X and y nibp
train_X_nibp,train_y_nibp=data_processing(reg_train_nibp,'NIBPmean')
test_X_nibp,test_y_nibp=data_processing(reg_test_nibp,'NIBPmean')


print("--- %s minutes ---" % round((time.time() - start_time)/60))

In [ ]:
#save processed data
pd.DataFrame(train_X_bis).to_csv('data\Data_model\\train_X_BIS.csv',index=False,header=False)
pd.DataFrame(train_X_mov).to_csv('data\Data_model\\train_X_MOV.csv',index=False,header=False)
pd.DataFrame(train_X_nibp).to_csv('data\Data_model\\train_X_NIBP.csv',index=False,header=False)

pd.DataFrame(train_y_bis).to_csv('data\Data_model\\train_y_BIS.csv',index=False,header=False)
pd.DataFrame(train_y_mov).to_csv('data\Data_model\\train_y_MOV.csv',index=False,header=False)
pd.DataFrame(train_y_nibp).to_csv('data\Data_model\\train_y_NIBP.csv',index=False,header=False)

pd.DataFrame(test_X_bis).to_csv('data\Data_model\\test_X_BIS.csv',index=False,header=False)
pd.DataFrame(test_X_mov).to_csv('data\Data_model\\test_X_MOV.csv',index=False,header=False)
pd.DataFrame(test_X_nibp).to_csv('data\Data_model\\test_X_NIBP.csv',index=False,header=False)

pd.DataFrame(test_y_bis).to_csv('data\Data_model\\test_y_BIS.csv',index=False,header=False)
pd.DataFrame(test_y_mov).to_csv('data\Data_model\\test_y_MOV.csv',index=False,header=False)
pd.DataFrame(test_y_nibp).to_csv('data\Data_model\\test_y_NIBP.csv',index=False,header=False)